Q3: Implementing an RNN for Text Generation using

Step 1: Import necessary libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import random
import sys

Step 2: Load Shakespeare dataset from TensorFlow

Step 3: Preprocess the text

Step 4: Create input sequences and targets

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')


vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])


seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


Step 5: Prepare the data for training

Step 6: Define the model

Step 7: Compile and train

In [3]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 512

model = Sequential([
    Embedding(vocab_size, embedding_dim),
    LSTM(rnn_units, return_sequences=True),
    Dense(vocab_size)
])

# Build the model with batch size
model.build(input_shape=(BATCH_SIZE, None))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(dataset, epochs=5)

Epoch 1/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 2.9029
Epoch 2/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 1.9547
Epoch 3/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 1.7203
Epoch 4/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 1.5882
Epoch 5/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 1.5117


Step 8: Text generation function

Step 9: Generate and display text

In [4]:
def generate_text(model, start_string, temperature=1.0):
    temp_model = Sequential([
        Embedding(vocab_size, embedding_dim),
        LSTM(rnn_units, return_sequences=True, stateful=True), # Make LSTM stateful
        Dense(vocab_size)
    ])
    temp_model.build(tf.TensorShape([1, None]))
    temp_model.set_weights(model.get_weights())

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    # Get the LSTM layer from the temp_model
    lstm_layer = temp_model.layers[1]

    # Reset the state of the LSTM layer
    lstm_layer.reset_states()

    for i in range(200):
        predictions = temp_model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

print("\nGenerated Text Sample:")
print(generate_text(model, start_string="To be, or not to be", temperature=0.2))

print("\nGenerated Text Sample:")
print(generate_text(model, start_string="To be, or not to be", temperature=0.8))


Generated Text Sample:
To be, or not to be the more.

CORIOLANUS:
What is the more than the world and the company.

PRINCE EDWARD:
What is the beat of the people to him.

MENENIUS:
What is the seatest the world of a country,
And we have been 

Generated Text Sample:
To be, or not to be,
But sees my born in this purisors too pleash:
And, you most made! I home one op!

CAPETLER:
If I see her a pentle parther, Deare and king I, foor gro,
When he do ever no good more graves,
Who dow ma
